In [ ]:
#!/usr/bin/env python3
# JetBot Dashboard - Verzia s WebSocket napojením na ROS
# Komunikace pomocí WebSocketů místo HTTP pollingu

from flask import Flask, render_template, Response, request, jsonify
from flask_socketio import SocketIO, emit
import threading
import time
import json
import socket
import os
import logging
import numpy as np
import cv2
import io
import traceback
import sys
import random
sys.path.append('/opt/ros/melodic/lib/python2.7/dist-packages')
import rospy
from std_msgs.msg import Float64, Int32

# Nastavení ROS prostředí
os.environ['ROS_MASTER_URI'] = 'http://localhost:11311'

# Nastavení loggeru
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('jetbot_dashboard')

# Kontrola dostupnosti ROS knihoven
ROS_AVAILABLE = False
try:
    import rospy
    from sensor_msgs.msg import Imu, LaserScan, CompressedImage
    from nav_msgs.msg import Odometry, OccupancyGrid
    from std_msgs.msg import Float64
    from geometry_msgs.msg import Twist
    ROS_AVAILABLE = True
    logger.info("ROS knihovny jsou dostupné")
except ImportError:
    logger.warning("ROS knihovny nejsou nainstalované. Dashboard bude mít omezenou funkčnost.")

# Konfigurace
WEB_PORT = 8080            # Port pro webový server
MJPEG_BOUNDARY = 'frame'   # Hranice pro MJPEG stream

# Vytvoření složky pro šablony, pokud neexistuje
TEMPLATE_FOLDER = 'templates'  # Složka pro HTML šablony
if not os.path.exists(TEMPLATE_FOLDER):
    os.makedirs(TEMPLATE_FOLDER)
    logger.info(f"Vytvořena složka pro šablony: {TEMPLATE_FOLDER}")

# Cesta k HTML šabloně
template_path = os.path.join(TEMPLATE_FOLDER, 'dashboard.html')

# Aplikace Flask a SocketIO
app = Flask(__name__, template_folder=TEMPLATE_FOLDER)
socketio = SocketIO(app, cors_allowed_origins="*", async_mode='threading')

# CORS hlavičky pro všechny odpovědi
@app.after_request
def add_cors_headers(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type')
    response.headers.add('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
    return response

# Globální proměnné pro ROS
cmd_vel_pub = None
ros_connected = False
ros_thread = None

# Aktuální data ze senzorů
sensor_data = {
    'imu': {
        'accel': {'x': 0, 'y': 0, 'z': 0},
        'gyro': {'x': 0, 'y': 0, 'z': 0},
        'orientation': {'x': 0, 'y': 0, 'z': 0, 'w': 0},
        'last_update': 0
    },
    'odom': {
        'position': {'x': 0, 'y': 0, 'z': 0},
        'orientation': {'x': 0, 'y': 0, 'z': 0, 'w': 0},
        'linear_velocity': {'x': 0, 'y': 0, 'z': 0},
        'angular_velocity': {'x': 0, 'y': 0, 'z': 0},
        'last_update': 0
    },
    'motors': {
        'left_velocity': 0,
        'right_velocity': 0,
        'last_update': 0
    },
    'scan': {
        'data': [],
        'min_distance': 0,
        'max_distance': 0,
        'point_count': 0,
        'last_update': 0
    }
}

# Kamerová data
camera_data = {
    'image': None,
    'last_update': 0,
    'quality': 80,
    'ros_stream': False
}

# Flag pro odesílání dat
emit_data_enabled = False

# =========== ROS Funkce a callbacky ===========

def imu_callback(msg):
    """Callback pro zpracování IMU dat z ROS"""
    try:
        # Akcelerometr
        sensor_data['imu']['accel']['x'] = msg.linear_acceleration.x
        sensor_data['imu']['accel']['y'] = msg.linear_acceleration.y
        sensor_data['imu']['accel']['z'] = msg.linear_acceleration.z
        
        # Gyroskop
        sensor_data['imu']['gyro']['x'] = msg.angular_velocity.x
        sensor_data['imu']['gyro']['y'] = msg.angular_velocity.y
        sensor_data['imu']['gyro']['z'] = msg.angular_velocity.z
        
        # Orientace
        sensor_data['imu']['orientation']['x'] = msg.orientation.x
        sensor_data['imu']['orientation']['y'] = msg.orientation.y
        sensor_data['imu']['orientation']['z'] = msg.orientation.z
        sensor_data['imu']['orientation']['w'] = msg.orientation.w
        
        # Aktualizace času
        sensor_data['imu']['last_update'] = time.time()
        
        # Odesílání dat přes WebSocket
        if emit_data_enabled:
            socketio.emit('imu_data', sensor_data['imu'])
        
        logger.debug("IMU data aktualizována")
    except Exception as e:
        logger.error(f"Chyba IMU dat: {e}")

def odom_callback(msg):
    """Callback pro zpracování odometrie z ROS"""
    try:
        # Pozice
        sensor_data['odom']['position']['x'] = msg.pose.pose.position.x
        sensor_data['odom']['position']['y'] = msg.pose.pose.position.y
        sensor_data['odom']['position']['z'] = msg.pose.pose.position.z
        
        # Orientace
        sensor_data['odom']['orientation']['x'] = msg.pose.pose.orientation.x
        sensor_data['odom']['orientation']['y'] = msg.pose.pose.orientation.y
        sensor_data['odom']['orientation']['z'] = msg.pose.pose.orientation.z
        sensor_data['odom']['orientation']['w'] = msg.pose.pose.orientation.w
        
        # Lineární rychlost
        sensor_data['odom']['linear_velocity']['x'] = msg.twist.twist.linear.x
        sensor_data['odom']['linear_velocity']['y'] = msg.twist.twist.linear.y
        sensor_data['odom']['linear_velocity']['z'] = msg.twist.twist.linear.z
        
        # Úhlová rychlost
        sensor_data['odom']['angular_velocity']['x'] = msg.twist.twist.angular.x
        sensor_data['odom']['angular_velocity']['y'] = msg.twist.twist.angular.y
        sensor_data['odom']['angular_velocity']['z'] = msg.twist.twist.angular.z
        
        # Aktualizace času
        sensor_data['odom']['last_update'] = time.time()
        
        # Odesílání dat přes WebSocket
        if emit_data_enabled:
            socketio.emit('odom_data', sensor_data['odom'])
        
        logger.debug("Odom data aktualizována")
    except Exception as e:
        logger.error(f"Chyba odometrie: {e}")

def left_motor_vel_callback(msg):
    """Callback pro zpracování rychlosti levého motoru z ROS"""
    try: 
        sensor_data['motors']['left_velocity'] = float(msg.data) / 100.0  # Adjust divisor as needed
        sensor_data['motors']['last_update'] = time.time()
        
        # Odesílání dat přes WebSocket
        if emit_data_enabled:
            socketio.emit('motors_data', sensor_data['motors'])
            
        logger.debug(f"Levý motor data aktualizována: {msg.data}")
    except Exception as e:
        logger.error(f"Chyba dat levého motoru: {e}")

def right_motor_vel_callback(msg):
    """Callback pro zpracování rychlosti pravého motoru z ROS"""
    try:
        sensor_data['motors']['right_velocity'] = float(msg.data) / 100.0  # Adjust divisor as needed
        sensor_data['motors']['last_update'] = time.time()
        
        # Odesílání dat přes WebSocket
        if emit_data_enabled:
            socketio.emit('motors_data', sensor_data['motors'])
            
        logger.debug(f"Pravý motor data aktualizována: {msg.data}")
    except Exception as e:
        logger.error(f"Chyba dat pravého motoru: {e}")

def scan_callback(msg):
    """Callback pro zpracování LIDAR dat z ROS"""
    try:
        # Extrakce dat ze zprávy
        angle_min = msg.angle_min
        angle_increment = msg.angle_increment
        ranges = msg.ranges
        
        # Konverze na formát pro HTML
        scan_data = []
        
        for i, distance in enumerate(ranges):
            if distance > 0 and not np.isinf(distance):  # Ignorování neplatných hodnot
                angle_rad = angle_min + i * angle_increment
                angle_deg = (angle_rad * 180 / 3.14159) % 360
                scan_data.append([angle_deg, float(distance) * 1000])  # Převod na stupně a mm
        
        # Omezení počtu bodů pro lepší výkon
        step = max(1, len(scan_data) // 36)  # Max 36 bodů
        scan_data = scan_data[::step]
        
        # Aktualizace dat skeneru
        sensor_data['scan']['data'] = scan_data
        sensor_data['scan']['point_count'] = len(scan_data)
        
        # Výpočet min/max vzdálenosti (v metrech)
        valid_distances = [d for d in ranges if d > 0 and not np.isinf(d)]
        if valid_distances:
            sensor_data['scan']['min_distance'] = min(valid_distances)
            sensor_data['scan']['max_distance'] = max(valid_distances)
        
        # Aktualizace času
        sensor_data['scan']['last_update'] = time.time()
        
        # Odesílání dat přes WebSocket
        if emit_data_enabled:
            # Odesíláme upravenou verzi pro nižší objem dat
            out_data = {
                'scan': scan_data,
                'point_count': sensor_data['scan']['point_count'],
                'min_distance': sensor_data['scan']['min_distance'],
                'max_distance': sensor_data['scan']['max_distance'],
                'last_update': sensor_data['scan']['last_update']
            }
            socketio.emit('scan_data', out_data)
            
        logger.debug("Scan data aktualizována")
    except Exception as e:
        logger.error(f"Chyba dat skeneru: {e}")

def camera_callback(msg):
    """Callback pro zpracování komprimovaného obrazu z ROS"""
    try:
        logger.debug("Přijata zpráva z kamerového topicu")
        
        # Uložení dat obrázku přímo z msg.data (už je to binární)
        camera_data['image'] = msg.data
        camera_data['last_update'] = time.time()
        camera_data['ros_stream'] = True
        
        # Pro WebSockety lze odeslat malý obrázek přímo, ale pro video
        # je lepší použít MJPEG stream kvůli výkonu
        # Nebudeme používat socketio.emit, protože velikost obrázku by byla moc velká
        
        logger.debug("Obrazová data úspěšně zpracována a uložena")
    except Exception as e:
        logger.error(f"Chyba zpracování obrazu z ROS: {e}")
        logger.error(traceback.format_exc())

def init_ros():
    """Inicializace ROS node a pripojení ke všem potřebným topikům"""
    global cmd_vel_pub, ros_connected
    
    if not ROS_AVAILABLE:
        logger.warning("ROS knihovny nejsou nainstalované, nelze inicializovat ROS")
        return False
    
    try:
        # Nastavení IP adresy tohoto ROS nodu (volitelné)
        os.environ['ROS_IP'] = get_ip_address()
        
        # Inicializace ROS node
        rospy.init_node('jetbot_dashboard', anonymous=True, disable_signals=True)
        logger.info("ROS node 'jetbot_dashboard' inicializován")
        
        # Přihlášení k topikům
        rospy.Subscriber('/imu', Imu, imu_callback)
        rospy.Subscriber('/odom_raw', Odometry, odom_callback)
        rospy.Subscriber('/motor/lvel', Int32, left_motor_vel_callback)
        rospy.Subscriber('/motor/rvel', Int32, right_motor_vel_callback)
        rospy.Subscriber('/scan', LaserScan, scan_callback)
        rospy.Subscriber('/csi_cam_0/image_raw/compressed', CompressedImage, camera_callback)
        
        # Logovanie dostupných topiců
        logger.info("Dostupné topicy: %s", str(rospy.get_published_topics()))
        
        # Vytvoření publisheru pro ovládání
        cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        
        # Nastavení stavu
        ros_connected = True
        
        logger.info("Úspěšně připojeno ke všem ROS topikům")
        
        # Kontrola dostupnosti topiců
        def check_subscriptions():
            time.sleep(10)
            logger.info(f"Kontrola ROS topiců po 10s:")
            logger.info(f"IMU poslední aktualizace: {sensor_data['imu']['last_update']}")
            logger.info(f"Odom poslední aktualizace: {sensor_data['odom']['last_update']}")
            logger.info(f"Motory poslední aktualizace: {sensor_data['motors']['last_update']}")
            logger.info(f"Scan poslední aktualizace: {sensor_data['scan']['last_update']}")
            logger.info(f"Kamera poslední aktualizace: {camera_data['last_update']}")
            logger.info(f"Kamera ROS stream: {camera_data['ros_stream']}")
            
            # Kontrola dostupnosti mapy
            try:
                map_msg = rospy.wait_for_message('/map', OccupancyGrid, timeout=1.0)
                logger.info(f"Mapa je dostupná, rozmery: {map_msg.info.width}x{map_msg.info.height}")
            except Exception as e:
                logger.warning(f"Mapa nie je dostupná: {e}")
        
        threading.Thread(target=check_subscriptions, daemon=True).start()
        
        # Povolení odesílání dat přes WebSocket
        global emit_data_enabled
        emit_data_enabled = True
        
        return True
    except Exception as e:
        logger.error(f"Chyba při inicializaci ROS: {e}")
        logger.error(traceback.format_exc())
        return False

def start_ros_spin():
    """Spustí rospy.spin() ve vlastním vlákně"""
    if not ROS_AVAILABLE or not ros_connected:
        return
    
    def ros_spin_thread():
        try:
            logger.info("Spouštím rospy.spin()")
            rospy.spin()
        except Exception as e:
            logger.error(f"Chyba v ROS spin vlákně: {e}")
    
    global ros_thread
    ros_thread = threading.Thread(target=ros_spin_thread, daemon=True)
    ros_thread.start()
    logger.info("ROS spin vlákno spuštěno")

# =========== Flask routy a WebSocket events ===========

@app.route('/')
def index():
    if os.path.exists(template_path):
        return render_template('dashboard.html')
    else:
        return """
        <html>
            <head><title>JetBot Dashboard - Chybí šablona</title></head>
            <body>
                <h1>Chybí HTML šablona!</h1>
                <p>Pro správnou funkci dashboardu je potřeba vytvořit HTML šablonu v:</p>
                <pre>{}</pre>
                <p>Aktuální pracovní adresář: {}</p>
                <p>Stav ROS: {}</p>
            </body>
        </html>
        """.format(template_path, os.getcwd(), "Dostupný" if ROS_AVAILABLE else "Nedostupný")

# ======= WebSocket události =======

@socketio.on('connect')
def handle_connect():
    logger.info("Klient se připojil přes WebSocket")
    # Odeslání informací o serveru při připojení
    emit('server_info', {
        'server_ip': get_ip_address(),
        'ros_connected': ros_connected,
        'camera_source': "ROS" if camera_data['ros_stream'] else "Nedostupná",
        'server_time': time.time(),
        'imu_updates': sensor_data['imu']['last_update'] > 0,
        'odom_updates': sensor_data['odom']['last_update'] > 0,
        'motors_updates': sensor_data['motors']['last_update'] > 0,
        'scan_updates': sensor_data['scan']['last_update'] > 0,
        'camera_updates': camera_data['last_update'] > 0
    })

@socketio.on('disconnect')
def handle_disconnect():
    logger.info("Klient se odpojil")

@socketio.on('get_server_info')
def handle_get_server_info():
    emit('server_info', {
        'server_ip': get_ip_address(),
        'ros_connected': ros_connected,
        'camera_source': "ROS" if camera_data['ros_stream'] else "Nedostupná",
        'server_time': time.time(),
        'imu_updates': sensor_data['imu']['last_update'] > 0,
        'odom_updates': sensor_data['odom']['last_update'] > 0,
        'motors_updates': sensor_data['motors']['last_update'] > 0,
        'scan_updates': sensor_data['scan']['last_update'] > 0,
        'camera_updates': camera_data['last_update'] > 0
    })

@socketio.on('get_imu_data')
def handle_get_imu_data():
    emit('imu_data', sensor_data['imu'])

@socketio.on('get_odom_data')
def handle_get_odom_data():
    emit('odom_data', sensor_data['odom'])

@socketio.on('get_motors_data')
def handle_get_motors_data():
    emit('motors_data', sensor_data['motors'])

@socketio.on('get_scan_data')
def handle_get_scan_data():
    output = {
        'scan': sensor_data['scan']['data'],
        'point_count': sensor_data['scan']['point_count'],
        'min_distance': sensor_data['scan']['min_distance'],
        'max_distance': sensor_data['scan']['max_distance'],
        'last_update': sensor_data['scan']['last_update']
    }
    emit('scan_data', output)

@socketio.on('control_robot')
def handle_control_robot(data):
    try:
        linear_x = float(data.get('linear_x', 0))
        angular_z = float(data.get('angular_z', 0))
        
        # Kontrola, zda jde o prioritní příkaz (např. příkaz zastavení)
        is_priority = bool(data.get('priority', False))
        
        # Omezení hodnot pro bezpečnost
        linear_x = max(-1.0, min(1.0, linear_x))
        angular_z = max(-1.0, min(1.0, angular_z))
        
        # Pokud ROS není připojen, vrátíme chybu
        if not ros_connected:
            emit('control_response', {'status': 'error', 'message': 'ROS není připojen'})
            return
        
        # Aplikace deadzone - malé hodnoty ignorujeme pro stabilitu
        if abs(linear_x) < 0.05:
            linear_x = 0.0
        if abs(angular_z) < 0.05:
            angular_z = 0.0
            
        # Detekce příkazu zastavení - prioritizujeme ho
        is_stop_command = (linear_x == 0.0 and angular_z == 0.0)
        
        # Kontrola frekvence příkazů - omezuje zahlcení
        current_time = time.time()
        if not hasattr(handle_control_robot, "last_command_time"):
            handle_control_robot.last_command_time = 0
        
        # Stop příkazy vždy zpracujeme okamžitě, ostatní příkazy omezíme na max 10 za sekundu (0.1s)
        if not is_stop_command and not is_priority and current_time - handle_control_robot.last_command_time < 0.1:
            # Příliš mnoho příkazů, ignorujeme tento
            return
        
        # Publikování zprávy přes ROS
        twist = Twist()
        twist.linear.x = linear_x
        twist.linear.y = 0.0
        twist.linear.z = 0.0
        twist.angular.x = 0.0
        twist.angular.y = 0.0
        twist.angular.z = angular_z
        
        # Příkazy zastavení posíláme s vyšší prioritou
        if is_stop_command:
            # Vyčištění případné fronty příkazů v ROS
            cmd_vel_pub.publish(twist)
            logger.debug("Robot zastaven")
        else:
            # Běžný příkaz pohybu
            cmd_vel_pub.publish(twist)
            handle_control_robot.last_command_time = current_time
        
        # Logování s omezením frekvence
        if not hasattr(handle_control_robot, "last_log_time"):
            handle_control_robot.last_log_time = 0
        
        if is_stop_command or current_time - handle_control_robot.last_log_time > 2.0:
            logger.debug(f"Příkaz: linear_x={linear_x}, angular_z={angular_z}")
            handle_control_robot.last_log_time = current_time
        
    except Exception as e:
        logger.error(f"Chyba při ovládání robota: {e}")
        emit('control_response', {'status': 'error', 'message': str(e)})
        
@socketio.on('start_pattern')
def handle_start_pattern(data):
    try:
        pattern = data.get('pattern', '')
        speed = float(data.get('speed', 0.3))
        size = float(data.get('size', 1.0))
        
        # Nový parameter pre priamy pohyb
        distance = float(data.get('distance', 1.0)) if 'distance' in data else None
        
        logger.info(f"Spúšťam vzor pohybu: {pattern}, rýchlosť: {speed}, veľkosť: {size}")
        
        # Pohyb po rovnej čiare
        if pattern == 'straight' and distance is not None:
            # Spustenie pohybu v samostatnom vlákne
            def execute_straight_movement():
                try:
                    # Kontrola či je ROS pripojený
                    if not ros_connected or not cmd_vel_pub:
                        socketio.emit('pattern_response', {
                            'status': 'error', 
                            'message': 'ROS nie je pripojený'
                        })
                        return
                    
                    # Nastavenie smeru podľa vzdialenosti
                    direction = 1 if distance >= 0 else -1
                    abs_distance = abs(distance)
                    
                    # Výpočet času potrebného na prejdenie vzdialenosti
                    time_required = abs_distance / speed
                    
                    # Vytvorenie správy pre pohyb
                    twist = Twist()
                    twist.linear.x = speed * direction  # Nastavenie rýchlosti a smeru
                    twist.angular.z = 0.0  # Bez zatáčania
                    
                    # Odoslanie príkazu pre pohyb
                    cmd_vel_pub.publish(twist)
                    
                    # Čakanie na prejdenie vzdialenosti
                    start_time = time.time()
                    progress = 0
                    
                    # Periodické aktualizácie pokroku
                    while progress < 100:
                        elapsed_time = time.time() - start_time
                        progress = min(100, (elapsed_time / time_required) * 100)
                        
                        # Posielanie aktualizácií každých 10%
                        if progress % 10 < 1:  # Posielame približne pri 0%, 10%, 20%, ...
                            socketio.emit('pattern_progress', {
                                'pattern': 'straight',
                                'progress': progress,
                                'elapsed_time': elapsed_time,
                                'remaining_time': max(0, time_required - elapsed_time)
                            })
                        
                        # Ak sme ešte nedosiahli 100%, počkáme 0.1s
                        if progress < 100:
                            time.sleep(0.1)
                    
                    # Zastavenie robota
                    twist.linear.x = 0.0
                    cmd_vel_pub.publish(twist)
                    
                    # Odoslanie správy o ukončení pohybu
                    socketio.emit('pattern_response', {
                        'status': 'success',
                        'message': f'Pohyb po rovnej čiare bol zastavený',
                        'pattern': 'straight',
                        'completed': True
                    })
                    
                    logger.info(f"Pohyb po rovnej čiare dokončený: vzdialenosť={distance} m, čas={time.time() - start_time:.2f} s")
                    
                except Exception as e:
                    logger.error(f"Chyba počas pohybu po rovnej čiare: {e}")
                    # Zastavenie robota v prípade chyby
                    if ros_connected and cmd_vel_pub:
                        twist = Twist()
                        twist.linear.x = 0.0
                        twist.angular.z = 0.0
                        cmd_vel_pub.publish(twist)
                    
                    socketio.emit('pattern_response', {
                        'status': 'error',
                        'message': f'Chyba: {str(e)}',
                        'pattern': 'straight'
                    })
            
            # Spustenie pohybu v samostatnom vlákne
            threading.Thread(target=execute_straight_movement, daemon=True).start()
            
            emit('pattern_response', {
                'status': 'success',
                'message': f'Vzor pohybu po rovnej čiare spustený: {abs(distance):.2f}m, {speed:.2f}m/s',
                'pattern': 'straight'
            })
            return
            
        # Existujúce vzory (kruh, štvorec) ostávajú nezmenené
        # Tu môžeš implementovať rôzne vzory pomocou samostatného vlákna
        # Pre jednoduchost budeme len vracať úspešnú odpoveď
        
        emit('pattern_response', {
            'status': 'success',
            'message': f'Vzor {pattern} bol spustený',
            'pattern': pattern,
            'speed': speed,
            'size': size
        })
    except Exception as e:
        logger.error(f"Chyba pri spúšťaní vzoru: {e}")
        emit('pattern_response', {'status': 'error', 'message': str(e)})

@socketio.on('stop_pattern')
def handle_stop_pattern():
    try:
        logger.info("Zastavujem vzor pohybu")
        
        # Zastavenie robota nastavením rýchlosti na 0
        if ros_connected and cmd_vel_pub:
            twist = Twist()
            twist.linear.x = 0.0
            twist.linear.y = 0.0
            twist.linear.z = 0.0
            twist.angular.x = 0.0
            twist.angular.y = 0.0
            twist.angular.z = 0.0
            cmd_vel_pub.publish(twist)
            
        emit('pattern_response', {
            'status': 'success',
            'message': 'Vzor pohybu bol zastavený'
        })
    except Exception as e:
        logger.error(f"Chyba pri zastavovaní vzoru: {e}")
        emit('pattern_response', {'status': 'error', 'message': str(e)})

@socketio.on('get_map_data')
def handle_get_map_data():
    try:
        # Preveriť, či je ROS pripojený
        if not ros_connected:
            emit('map_data', {'error': 'ROS nie je pripojený'})
            return
        
        # Bezpečnejší spôsob získavania mapy - pomocou subscribera
        map_data = None
        map_received = threading.Event()
        
        def map_callback(msg):
            nonlocal map_data
            try:
                # Konverzia do formátu pre JavaScript
                # Optimalizácia: limitujeme množstvo dát - posielame iba každý 2. bod pri mapách väčších ako 500x500
                decimation_factor = 1
                data_array = list(msg.data)
                
                if msg.info.width > 500 or msg.info.height > 500:
                    decimation_factor = 2
                    # Zredukované rozlíšenie - každý N-tý pixel
                    reduced_width = msg.info.width // decimation_factor
                    reduced_height = msg.info.height // decimation_factor
                    reduced_data = []
                    
                    for y in range(0, msg.info.height, decimation_factor):
                        for x in range(0, msg.info.width, decimation_factor):
                            idx = y * msg.info.width + x
                            if idx < len(data_array):
                                reduced_data.append(data_array[idx])
                    
                    data_array = reduced_data
                    logger.info(f"Redukovaná mapa: {reduced_width}x{reduced_height}, dekadický faktor: {decimation_factor}")
                
                map_data = {
                    'width': msg.info.width // decimation_factor,
                    'height': msg.info.height // decimation_factor,
                    'resolution': msg.info.resolution * decimation_factor,
                    'origin_x': msg.info.origin.position.x,
                    'origin_y': msg.info.origin.position.y,
                    'decimation_factor': decimation_factor,
                    'data': data_array
                }
                map_received.set()
            except Exception as e:
                logger.error(f"Chyba pri spracovaní mapy: {e}")
        
        # Vytvoríme dočasného subscribera
        map_sub = rospy.Subscriber('/map', OccupancyGrid, map_callback, queue_size=1)
        
        # Počkáme maximálne 3 sekundy na prijatie mapy (skrátené z 5s)
        if map_received.wait(timeout=10.0):
            logger.info(f"Mapa úspešne získaná: {map_data['width']}x{map_data['height']}")
            emit('map_data', map_data)
        else:
            logger.warning("Timeout pri čakaní na mapu")
            emit('map_data', {'error': 'Timeout pri získavaní mapy'})
        
        # Odhlásenie subscribera
        map_sub.unregister()
        
    except Exception as e:
        logger.error(f"Chyba pri získavaní mapy: {e} - {traceback.format_exc()}")
        emit('map_data', {'error': str(e)})

@socketio.on('reset_map')
def handle_reset_map():
    try:
        import subprocess
        # Zastaviť a znovu spustiť gmapping
        subprocess.run(['rosnode', 'kill', '/slam_gmapping'])
        time.sleep(2)  # Počkaj kým sa uzavrie
        subprocess.Popen(['rosrun', 'gmapping', 'slam_gmapping', 'scan:=/scan'])
        
        emit('map_response', {'status': 'success', 'message': 'Mapa bola resetovaná'})
    except Exception as e:
        logger.error(f"Chyba pri resetovaní mapy: {e}")
        emit('map_response', {'status': 'error', 'message': str(e)})

@socketio.on('save_map')
def handle_save_map():
    try:
        import subprocess
        import datetime
        
        # Vytvor adresár pre mapy, ak neexistuje
        maps_dir = 'maps'
        if not os.path.exists(maps_dir):
            os.makedirs(maps_dir)
        
        # Vytvor meno súboru s časovou pečiatkou
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        map_name = f"map_{timestamp}"
        map_path = os.path.join(maps_dir, map_name)
        
        # Spusti map_saver na uloženie mapy
        subprocess.run(['rosrun', 'map_server', 'map_saver', '-f', map_path])
        
        emit('map_response', {
            'status': 'success',
            'message': 'Mapa bola uložená',
            'filename': f"{map_name}.pgm"
        })
    except Exception as e:
        logger.error(f"Chyba pri ukladaní mapy: {e}")
        emit('map_response', {'status': 'error', 'message': str(e)})

# =========== Video stream - zůstává jako HTTP endpoint ===========

# Generátor snímků pro MJPEG stream
def generate_frames():
    last_frame_time = time.time()
    
    while True:
        try:
            # Omezení FPS pro snížení zatížení
            current_time = time.time()
            elapsed = current_time - last_frame_time
            if elapsed < (1.0 / 30): 
                time.sleep((1.0 / 30) - elapsed)
            last_frame_time = time.time()
            
            # Pokud máme k dispozici obraz z ROS kamery
            if camera_data['image'] is not None and camera_data['ros_stream']:
                logger.debug("Odesílání obrazu z ROS streamu")
                frame = camera_data['image']
                yield (b'--' + MJPEG_BOUNDARY.encode() + 
                       b'\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            else:
                # Pokud nemáme obraz z kamery, zobrazíme prázdný obraz s informací
                img = np.zeros((240, 320, 3), np.uint8)  # Černé pozadí
                cv2.putText(img, "Kamera nedostupná", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                ret, jpeg = cv2.imencode('.jpg', img, [cv2.IMWRITE_JPEG_QUALITY, 70])
                frame = jpeg.tobytes()
                yield (b'--' + MJPEG_BOUNDARY.encode() + 
                       b'\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        except Exception as e:
            logger.error(f"Chyba při generování snímku: {e}")
            time.sleep(0.5)

# MJPEG stream - Ponecháváme jako HTTP endpoint, protože video přes WebSocket by bylo neefektivní
@app.route('/video_feed')
def video_feed():
    try:
        logger.info("Video feed request")
        response = Response(generate_frames(),
                      mimetype='multipart/x-mixed-replace; boundary=' + MJPEG_BOUNDARY)
        # Přidání hlaviček pro správnou funkci MJPEG streamu
        response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
        response.headers['Pragma'] = 'no-cache'
        response.headers['Expires'] = '0'
        return response
    except Exception as e:
        logger.error(f"Chyba při vytváření video streamu: {e}")
        return f"Chyba video streamu: {e}"

# =========== Pomocné funkce ===========

# Funkce pro získání IP adresy
def get_ip_address():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

# Spuštění Flask a SocketIO serveru
def start_web_server():
    def run_server():
        try:
            logger.info(f"Spouštění webového serveru na portu {WEB_PORT}...")
            # Spuštění pomocí SocketIO místo app.run()
            socketio.run(app, host='0.0.0.0', port=WEB_PORT, debug=False, use_reloader=False, allow_unsafe_werkzeug=True)
        except Exception as e:
            logger.error(f"Chyba při spouštění serveru: {e}")
    
    # Vytvoření a spuštění vlákna serveru
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    
    # Krátké čekání na spuštění serveru
    time.sleep(1)
    
    # Výpis informací o serveru
    server_ip = get_ip_address()
    logger.info(f"Webový dashboard běží na http://{server_ip}:{WEB_PORT}")
    
    return f"http://{server_ip}:{WEB_PORT}"

# Hlavní funkce pro spuštění dashboardu
def start_dashboard():
    """
    Spustí dashboard pro JetBot s WebSocket napojením na ROS
    """
    try:
        # Pokud je ROS dostupný, zkusíme se připojit
        if ROS_AVAILABLE:
            if init_ros():
                # Spuštění rospy.spin() ve vlastním vlákně
                start_ros_spin()
                logger.info("ROS úspěšně inicializován")
            else:
                logger.warning("Nepodařilo se inicializovat ROS")
        else:
            logger.warning("ROS není dostupný - dashboard bude mít omezenou funkčnost")
        
        # Spuštění webového serveru
        url = start_web_server()
        logger.info(f"Webový server běží na: {url}")
        
        return f"Dashboard je spuštěn na {url}. Přístup přes webový prohlížeč."
    
    except Exception as e:
        logger.error(f"Chyba při spouštění dashboardu: {e}")
        return f"Chyba při spouštění dashboardu: {e}"

# Spuštění dashboardu
if __name__ == "__main__":
    try:
        print(start_dashboard())
        
        # Udržet hlavní vlákno běžící
        while True:
            time.sleep(1)
            
    except KeyboardInterrupt:
        logger.info("Ukončení aplikace uživatelem (Ctrl+C)")
    except Exception as e:
        logger.error(f"Chyba při běhu aplikace: {e}")
        traceback.print_exc()